In [107]:
import pandas as pd
import numpy as np
import time, os, datetime
from datetime import date, datetime
from datetime import datetime
from dateutil import relativedelta

## Init

In [7]:
df_test = pd.read_csv('./data/ejemplo_de_solucion.csv')
df_train = pd.read_csv('./data/postulaciones/postulaciones_train.csv')

In [108]:
df_aDetalle = pd.read_csv('./data/avisos/avisos_detalle.csv')
df_aOnline = pd.read_csv('./data/avisos/avisos_online.csv')
df_postEdu = pd.read_csv('./data/postulantes/postulantes_educacion.csv')
df_postGenero = pd.read_csv('./data/postulantes/postulantes_genero_edad.csv')

In [9]:
len(df_test.idpostulante.unique())


156232

In [10]:
df_test.loc[df_test['idpostulante']=='ZaO5']

,idaviso,idpostulante
0,1112427970,ZaO5
156232,1112094756,ZaO5


## Comienzo fe

In [11]:
#Elimino filas repetidas
df_postEdu=df_postEdu.drop_duplicates(subset='idpostulante')
df_postGenero=df_postGenero.drop_duplicates(subset='idpostulante')

In [104]:
# Acumulo informacion de los postulantes
df_postulante = df_postGenero.merge(df_postEdu, on='idpostulante')
df_postulante

,idpostulante,fechanacimiento,sexo,nombre,estado
0,NM5M,1970-12-03,FEM,Secundario,Graduado
1,5awk,1962-12-04,FEM,Universitario,Graduado
2,ZaO5,1978-08-10,FEM,Terciario/Técnico,Graduado
3,NdJl,1969-05-09,MASC,Posgrado,En Curso
4,eo2p,1981-02-16,MASC,Secundario,Graduado
...,...,...,...,...,...
447904,GNZr6xw,1997-08-17,MASC,Universitario,En Curso
447905,OqrJ8zv,1998-06-23,FEM,Terciario/Técnico,Graduado
447906,X95VzpK,1998-02-25,MASC,Universitario,En Curso
447907,QNr9A2N,1992-08-01,MASC,Master,En Curso


In [30]:
nacimiento =df_postulante[:1].fechanacimiento.values[0]
nacimiento

'1970-12-03'

In [48]:
ts_1= datetime.strptime(nacimiento, '%Y-%m-%d')
ts_2= datetime.strptime('2018-04-01', '%Y-%m-%d')
difference = relativedelta.relativedelta(ts_2, ts_1)
difference.years

47

In [53]:
df_postulante['edad'] = relativedelta.relativedelta(ts_2, datetime.strptime(df_postulante['fechanacimiento'].values[0], '%Y-%m-%d')).years

In [125]:
def diff_edad(row):
    return relativedelta.relativedelta(ts_2, datetime.strptime(row['fechanacimiento'], '%Y-%m-%d')).years

In [127]:
#diff_edad(df_postulante[:1]['fechanacimiento'])

In [175]:
# fechanacimiento tiene 9391 nan
moda_fechanacimiento= df_postulante.fechanacimiento.mode()
moda_fechanacimiento
df_postulante = df_postulante.replace(np.nan, '1992-04-01', regex=True)

0    1992-04-01
dtype: object

In [182]:
df_postulante['edad'] = df_postulante.apply(diff_edad, axis=1)

In [195]:
moda_edad= df_postulante.edad.mode()
print(moda_edad)
df_postulante.edad.describe()

0    26
dtype: int64


count    447909.000000
mean         30.431217
std          16.869111
min           7.000000
25%          24.000000
50%          28.000000
75%          35.000000
max        2016.000000
Name: edad, dtype: float64

In [238]:
 tipo_edad = []
 for row in df_postulante['edad']:
     if row < 25 :           tipo_edad.append('cat1')
     elif 25 <= row <= 35:   tipo_edad.append('cat2')
     else:                   tipo_edad.append('cat3')
            
 df_postulante['tipo_edad'] = tipo_edad
 df_postulante.head()

,idpostulante,fechanacimiento,sexo,nombre,estado,edad,tipo_edad
0,NM5M,1970-12-03,FEM,Secundario,Graduado,47,cat3
1,5awk,1962-12-04,FEM,Universitario,Graduado,55,cat3
2,ZaO5,1978-08-10,FEM,Terciario/Técnico,Graduado,39,cat3
3,NdJl,1969-05-09,MASC,Posgrado,En Curso,48,cat3
4,eo2p,1981-02-16,MASC,Secundario,Graduado,37,cat3


In [239]:
# Datos de los postulantes filtrados y consolidados con Genero y Educacion.
df_postulante.to_csv("./data/postulantes/postulantes_consolidado.csv", index=False, header=True)

In [14]:
print(df_aDetalle.shape, df_aOnline.shape)

(24950, 11) (18362, 3)


In [113]:
# Me quedo con los avisos/postulaciones que tiene fecha de inicio y fin, las otras las elimino.
df_avisos = df_aDetalle.merge(df_aOnline, on='idaviso')
# Elimino idPais: Son todos 1
df_avisos = df_avisos.drop(['idpais','descripcion', 'ciudad', 'mapacalle'], axis=1)

In [114]:
df_avisos.head()

,idaviso,titulo,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,online_desde,online_hasta
0,8725750,VENDEDOR/A PROVINCIA DE SANTA FE,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Comercial,VENTOR,2018-01-15,2018-02-10
1,17903700,Enfermeras,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Salud,Farmacias Central Oeste,2018-03-20,2018-04-17
2,1000610287,CHOFER DE CAMIONETA BAHIA BLANCA - PUNTA ALTA,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Transporte,Wurth Argentina S.A,2018-01-15,2018-03-17
3,1001135716,Vendedor Viajante TUCUMAN/SANTIAGO DEL ESTERO,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Ventas,Wurth Argentina S.A,2018-01-15,2018-04-16
4,1001326344,Vendedor Viajante RECONQUISTA/AVELLANEDA,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Ventas,Wurth Argentina S.A,2018-01-15,2018-03-27


In [115]:
# Avisos filtrados y con fecha de inici y fin
df_avisos.to_csv("./data/avisos/avisos_filtrados.csv", index=False, header=True)

## Revision de datos train vs test

In [17]:
idpost_train = df_train.idpostulante
idpost_test = df_test.idpostulante

In [18]:
idpost_train_set = set(idpost_train.unique().tolist())
idpost_test_set = set(idpost_test.unique().tolist())
print(len(idpost_train_set), len(idpost_test_set))

302787 156232


In [19]:
test_unique = (idpost_test_set - idpost_train_set)
test_intersect = (idpost_test_set - test_unique)
print("unicos de test: ",len(test_unique ))
print("compartidos con train: ", len(test_intersect))

unicos de test:  41204
compartidos con train:  115028


## Me quedo con los postulantes que estan en TRAIN unicamente

In [20]:
idpost_full = df_postulante.idpostulante
print(len(idpost_full))

447909


In [21]:
# TRAIN SIN REPETIDOS
len(df_train.drop_duplicates(subset='idpostulante'))

302787

In [24]:
df_postulantes_justInTrain = df_postulante.merge(df_train.drop_duplicates(subset='idpostulante'), on='idpostulante')
df_postulantes_justInTrain = df_postulantes_justInTrain[['idpostulante','fechanacimiento','sexo','nombre','estado']]
len(df_postulantes_justInTrain)

283163

In [26]:
# Postulantes que SOLO estan en Train: justInTrain
df_postulantes_justInTrain.to_csv("./data/postulantes/postulantes_justInTrain.csv", index=False, header=True)

## Cantidad de postulaciones de los usuarios en TRAIN

In [3]:
# Train dataset con mas de 21 postulaciones
#df_train[:100].loc[df_train.groupby(['idpostulante']).size().reset_index(name='counts')['counts']>21]

In [30]:
df_train_group= df_train.groupby(['idpostulante']).size().reset_index(name='counts')
df_train_group.sort_values('counts',ascending=False) 

,idpostulante,counts
64995,8MaQjP3,3945
54835,6rQdqjl,3614
35484,5Mwjak,3545
121625,KBzm0ZX,3010
213480,axmjOE,2850
...,...,...
142425,NzjDkRx,1
142422,NzjDGpB,1
266214,pzdEJPb,1
266216,pzdEW6N,1


In [31]:
df_train_group['counts'].describe()

count    302787.000000
mean         21.363827
std          49.877822
min           1.000000
25%           3.000000
50%           8.000000
75%          21.000000
max        3945.000000
Name: counts, dtype: float64

In [33]:
len(df_train_group.loc[df_train_group['counts']>= 100])

11343

In [34]:
df_train_filtered = df_train.merge(df_train_group, on='idpostulante')
print(len(df_train_filtered))

6468689


In [35]:
df_train_filtered['counts'] = df_train_filtered['counts'].astype('int')
df_train_filtered = df_train_filtered.loc[df_train_filtered['counts']>=300]
len(df_train_filtered)

676572

In [36]:
len(df_train_filtered.groupby(['idpostulante']))

1363

In [37]:
df_train_filtered = df_train_filtered[['idaviso','idpostulante','fechapostulacion']] 
df_train_filtered.head()

,idaviso,idpostulante,fechapostulacion
4272,1112266459,eeoNQ4,2018-01-19 13:16:52
4273,1112266448,eeoNQ4,2018-01-19 13:18:55
4274,1112230347,eeoNQ4,2018-01-19 13:21:29
4275,1112186063,eeoNQ4,2018-01-20 20:02:32
4276,1111944699,eeoNQ4,2018-01-20 20:17:18


In [21]:
# Postulantes que tienen mas de 100 publicaciones
df_train_filtered.to_csv("./data/postulantes/postulantes_train_filtered.csv", index=False, header=True)

## Generacion Random
### Para los users/idpostulante que estan en Test y no en Train se le asigna un idaviso random, que cumple dos condiciones:
- Es un aviso valido en el mes de Abril 2018
- Es un aviso con al menos 2 postulaciones

In [ ]:
def intersection(lst1, lst2): 
    return list(set(lst1) & set(lst2)) 

In [ ]:
df_avisos_mas_vistos= df_train.groupby(['idaviso']).size().reset_index(name='counts')
df_avisos_mas_vistos.sort_values('counts',ascending=True) 

In [ ]:
avisos_mas2 = df_avisos_mas_vistos.loc[df_train_group['counts']>= 2].sort_values('counts',ascending=False).idaviso.tolist() 

In [ ]:
abril_y_mas2= intersection(avisos_mas2,avisos_abril_list)
print(len(abril_y_mas2))
print(len(avisos_abril_list))
print(len(abril_y_mas2))

In [ ]:
1111556097 in avisos_abril_list

In [ ]:
# Random con AVISOS DE ABRIL 2018.
random.seed(12)
scored_list_unknow = []
for user_unknown in tqdm_notebook(users_unknown):
    for index in range(10):
        user_aviso=dict()
        user_aviso["idaviso"] = random.choices(abril_y_mas2)[0]
        user_aviso["idpostulante"] = user_unknown
        scored_list_unknow.append(user_aviso)
#scored_list_unknow

In [ ]:
# Creo dataframe con la lista random de los postulantes que estan solo en test
df_modelo_unknow = pd.DataFrame(scored_list_unknow)
df_modelo_unknow.head()

In [ ]:
# Appendeo un DF a otro DF
#df_modelo_final = df_modelo_intermedio.append(df_modelo_unknow)

In [ ]:
df_xxxx.to_csv("./xxxx.csv", index=False)